# Smart Connector for Thermostat API

This notebook acts as a translation layer between the thermostat API and a knowledge engine. It serves as a knowledge base and registers with the knowledge engine using functions `register_post_knowledge_interaction` and `register_react_knowledge_interaction`. In turn, it receives an ID for each interaction that is necessary to post or handle a react.

There are two types of interactions: POST/REACT and ASK/ANSWER. When the graph pattern of this smart connector's post_knowledge_interaction gets matched by the knowledge engine with the graph pattern of another knowledge base's react_knowledge_interaction, the post side sends bindings that are then received and handled by the react side. ASK/ANSWER interactions will be explained in further notebooks.

This smart connector has four main knowledge interactions:

## `(POST)` "post-temp-measurements" 

This knowledge interaction is used to post live temperature measurements to the knowledge engine. It takes the current temperature as measured by the thermostat API, constructs bindings describing the measurement, and sends them to the knowledge engine. This interaction runs in a separate thread in the background.

## `(REACT)`  "set-desired-temp" 

This knowledge interaction changes the desired temperature of the thermostat. When triggered, it calls a function that takes a list of bindings as an argument. This function logs a message indicating that it is changing the desired temperature to the value specified in the binding. Then, it sends a PUT request to the thermostat API's `/desired_temperature` endpoint with the desired temperature value as a query parameter to update the desired temperature of the thermostat. This interaction is handled by the main thread, which is occupied by a handle_loop that handles react KIs.

## `(POST)` "post-humidity-measurements"

This knowledge interaction is used to post live humidity measurements to the knowledge engine. It takes the current humidity as measured by the thermostat API, constructs bindings describing the measurement, and sends them to the knowledge engine. This interaction runs in a separate thread in the background.

## `(REACT)`  "set-desired-humidity"

This knowledge interaction changes the desired humidity of the thermostat. When triggered, it calls a function that takes a list of bindings as an argument. This function logs a message indicating that it is changing the desired humidity to the value specified in the binding. Then, it sends a PUT request to the thermostat API's `/desired_humidity` endpoint with the desired humidity value as a query parameter to update the desired humidity of the thermostat. This interaction is handled by the main thread, which is occupied by a handle_loop that handles react KIs.

All four KIs have their own RDF graph patterns that describe them using the SAREF ontology. These graph patterns are string constants and are hardcoded when registering them with the knowledge engine. After that, only bindings are set and received. These graph patterns can be used by the knowledge engine to interact with this smart connector and other similar knowledge bases.

In [ ]:
# ThermostatSC2

In [ ]:
import datetime
import logging
import random
import time
import uuid

import helpers as sp
from utils import *

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("ThermostatSC")

In [ ]:
THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"
# prefixes of ontologies used in the graph patterns
PREFIXES = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "saref": "https://w3id.org/saref#",
    "xsd": "http://www.w3.org/2001/XMLSchema#",
}
# temp measurement graph pattern
TEMP_MEAS_GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
                        ?meas saref:hasValue ?temp .
                        ?meas saref:isMeasuredIn saref:TemperatureUnit .
                        ?meas saref:hasTimestamp ?timestamp .
                        ?meas saref:isMeasurementOf ?room_id .
                        ?meas saref:relatesToProperty saref:Temperature .
                        ?meas saref:measurementMadeBy ?device_id ."""
# graph pattern describing the change of the ?desired_temp setting of the thermostat
TEMP_SETTING_GRAPH_PATTERN = """?setting rdf:type saref:SetLevelCommand .
                            ?setting saref:hasValue ?desired_temp .
                            ?setting saref:isMeasuredIn saref:TemperatureUnit .
                            ?setting saref:hasTimestamp ?timestamp .
                            ?setting saref:isCommandOf ?room_id .
                            ?setting saref:relatesToProperty saref:Temperature .
                            ?setting saref:commandIssuedBy ?device_id .
                            """
# humidity measurement graph pattern
HUMIDITY_MEAS_GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
                        ?meas saref:hasValue ?humidity .
                        ?meas saref:isMeasuredIn saref:HumidityUnit .
                        ?meas saref:hasTimestamp ?timestamp .
                        ?meas saref:isMeasurementOf ?room_id .
                        ?meas saref:relatesToProperty saref:Humidity .
                        ?meas saref:measurementMadeBy ?device_id ."""
# graph pattern describing the change of the ?desired_humidity setting of the thermostat
HUMIDITY_SETTING_GRAPH_PATTERN = """?setting rdf:type saref:SetLevelCommand .
                            ?setting saref:hasValue ?desired_humidity .
                            ?setting saref:isMeasuredIn saref:HumidityUnit .
                            ?setting saref:hasTimestamp ?timestamp .
                            ?setting saref:isCommandOf ?room_id .
                            ?setting saref:relatesToProperty saref:Humidity .
                            ?setting saref:commandIssuedBy ?device_id .
                            """

In [ ]:
def get_temp_now():
    return requests.get(THERMOSTAT_API_URL + "/actual_temperature").json()

In [ ]:
def temp_measurement_loop(post_temp_ki_id, kb_id, ke_endpoint):
    value = get_temp_now()
    logger.info(
        f"attempting to post temp measurement of {value} units at {sp.get_timestamp_now()}"
    )
    post(
        [
            {
                "meas": f"<{THERMOSTAT_API_URL}/measurements/{str(uuid.uuid4())}>",
                "temp": f"{value}",
                "timestamp": f'"{sp.get_timestamp_now()}"',  # ISO 8601 format
                "room_id": f'"{THERMOSTAT_API_URL}/rooms/1"',
                "device_id": f'"{THERMOSTAT_API_URL}/devices/1"',
            }
        ],
        post_temp_ki_id,
        kb_id,
        ke_endpoint,
    )
    logger.info(
        f"Published temperature measurement of {value} units at {sp.get_timestamp_now()}"
    )

    time.sleep(5)

In [ ]:
def get_humidity_now():
    return requests.get(THERMOSTAT_API_URL + "/actual_humidity").json()

In [ ]:
def humidity_measurement_loop(post_humidity_ki_id, kb_id, ke_endpoint):
    value = get_humidity_now()
    logger.info(
        f"attempting to post humidity measurement of {value} units at {sp.get_timestamp_now()}"
    )

    post(
        [
            {
                "meas": f"<{THERMOSTAT_API_URL}/measurements/{str(uuid.uuid4())}>",
                "humidity": f"{value}",
                "timestamp": f'"{sp.get_timestamp_now()}"',  # ISO 8601 format
                "room_id": f'"{THERMOSTAT_API_URL}/rooms/1"',
                "device_id": f'"{THERMOSTAT_API_URL}/devices/1"',
            }
        ],
        post_humidity_ki_id,
        kb_id,
        ke_endpoint,
    )
    logger.info(
        f"Published humidity measurement of {value} units at {sp.get_timestamp_now()}"
    )

    time.sleep(5)

In [ ]:
def handle_react_change_desired_temp(bindings):
    for binding in bindings:
        logger.info(f"Changing desired temperature to: {binding['desired_temp']}")
        # change desired temp
        requests.put(
            THERMOSTAT_API_URL
            + "/desired_temperature"
            + f'?desired_temperature={int(binding["desired_temp"])}'
        )
    return []

In [ ]:
def handle_react_change_desired_humidity(bindings):
    for binding in bindings:
        logger.info(f"Changing desired humidity to: {binding['desired_humidity']}")
        # change desired temp
        requests.put(
            THERMOSTAT_API_URL
            + "/desired_humidity"
            + f'?desired_humidity={int(binding["desired_humidity"])}'
        )
    return []

In [ ]:
import threading


def start_sensor_kb(
    kb_id, kb_name, kb_description, ke_endpoint, post_live_measurements=True
):
    # delete in case allready exists
    delete_knowledge_base(kb_id, ke_endpoint)

    # register kb
    register_knowledge_base(kb_id, kb_name, kb_description, ke_endpoint)

    post_temp_ki_id = register_post_knowledge_interaction(
        TEMP_MEAS_GRAPH_PATTERN,
        None,
        "post-temp-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    post_humidity_ki_id = register_post_knowledge_interaction(
        HUMIDITY_MEAS_GRAPH_PATTERN,
        None,
        "post-humidity-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    temp_react_ki_id = register_react_knowledge_interaction(
        TEMP_SETTING_GRAPH_PATTERN,
        None,
        "set-desired-temp",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    humidity_react_ki_id = register_react_knowledge_interaction(
        HUMIDITY_SETTING_GRAPH_PATTERN,
        None,
        "set-desired-humidity",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    def measurement_loop(post_temp_ki_id, post_humidity_ki_id, kb_id, ke_endpoint):
        while True:
            temp_measurement_loop(post_temp_ki_id, kb_id, ke_endpoint)
            humidity_measurement_loop(post_humidity_ki_id, kb_id, ke_endpoint)

    measurement_thread = threading.Thread(
        target=measurement_loop,
        args=(
            post_temp_ki_id,
            post_humidity_ki_id,
            kb_id,
            ke_endpoint,
        ),
    )

    if post_live_measurements:
        measurement_thread.start()

    # Start the start_handle_loop function in a separate thread
    start_handle_loop(
        {
            temp_react_ki_id: handle_react_change_desired_temp,
            humidity_react_ki_id: handle_react_change_desired_humidity,
        },
        kb_id,
        ke_endpoint,
    )

In [ ]:
start_sensor_kb(
    "http://example.org/thermostatSC",
    "Thermostat_1",
    "A thermostat",
    "http://knowledge_engine:8280/rest/",
)